# Example: Prospect Theory, Basket Option

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')
import matplotlib
matplotlib.rcParams.update({'font.size': 22})
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp
import scipy
import seaborn
from tqdm import tqdm
import yfinance as yf
from scipy.stats import norm
from optimization_functions import *
from functions_multidim import *
%load_ext autoreload
%autoreload 2
%aimport optimization_functions
%aimport functions_multidim   


Download Stock Prices and create Training Periods

In [ ]:
stocks = ["GOOGL","EBAY","AMZN","MSFT","AAPL"]
d = len(stocks)
Returns_train_tf_dict = {}

In [ ]:
for s in stocks:
    S_train = yf.download(s, start="2010-01-02", end="2020-02-01")
    S_train = S_train.dropna()
    S_train = S_train.reset_index()
    S_train = S_train[["Date","Close"]]
    
    Returns_train = S_train["Close"].diff() / S_train["Close"].shift(1)
    Returns_train = Returns_train.dropna()
    # A Tensorflow Adaption
    Returns_train_tf_dict[s] = tf.reshape(Returns_train,(1,len(Returns_train)))
    
# Create a d dimensional vector
Returns_train_tf = tf.concat([Returns_train_tf_dict[s] for s in stocks],0)

Test Period 1: Feb 2020 - May 2020

# Description of the Algorithm

For $t = T-1, \cdots , 1$ :


For $j = 1,\dots, N_{iterations}$:

Sample $(\omega_1,\dots,\omega_t)=\omega^t$

Sample $(a_0,\dots,a_{t-1})=a^t$

For $k = 1,\dots, N_{measures}$:

Sample next states
$w^{t+1, (k),(i)} \sim \mathbb{P}_k \in \mathcal{P}_t(\omega^t)$ for $i= 1,\dots,N_{MC}$

<!-- Compute $\tilde{J}_t(\omega,a^t, \tilde{a}) = \min_k \frac{1}{N}\sum_{i=1} \tilde{\Psi}_{t+1}((\omega^t,a^t, \tilde{a}))$
Maximize w.r.t. $\widetilde{a}$:

$\widetilde{a}_t(\omega^t,a^t)=argmax_{\widetilde{a}} \tilde{J}_t(\omega,a^t, \tilde{a})$

Approximate

$\tilde{\Psi}_t(\omega^t, a^t):= \tilde{J}_t(\omega,a^t, \widetilde{a}_t(\omega^t,a^t))$ -->

The optimization step is:

1.)

Maximize
$$
\sup_{\widetilde{a} \in A}\inf_{ P \in \mathcal{P}_t(\omega^t)} E_P \left[\Psi_{t+1}\left(\omega^t \otimes_t \cdot, (a^{t},\widetilde{a})\right)\right]= \Psi_t(\omega^t,a^t)
$$
 w.r.t. parameters of $\tilde{a}(\omega^t,a^t)$:

 Alternative:

 By using a dual approach we have in the special case of a Wasserstein distance
 $$
\Psi_t(\omega^t,a^t)=\sup_{\widetilde{a} \in A}\inf_{ P \in \mathcal{P}_t(\omega^t)} E_P \left[\Psi_{t+1}\left(\omega^t \otimes_t \cdot, (a^{t},\widetilde{a})\right)\right]
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0} E_{\widehat{P}} \left[\inf_{z} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z, (a^{t},\widetilde{a})\right)+\lambda \|\cdot -z\|\right\}\right] - \lambda \varepsilon
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0} \frac{1}{N_{MC}} \sum_{i=1}^{N_{MC}} \left[\min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|x_i -z_j\|\right\}\right] - \lambda \varepsilon
$$
$$
= \sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0}  \sum_{i=t}^{N-1} \left[\pi_i(\omega^t) \min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|\mathcal{R}_{i+1} -z_j\|\right\}\right] - \lambda \varepsilon
$$
2.)


Minimize
$$
\sum_{b=1}^B \left(\tilde{\Psi}_t(\omega^t_b, a^t_b)- \left(\sup_{\widetilde{a} \in A}\sup_{ \lambda \geq 0}  \sum_{i=t}^{N-1} \left[\pi_i(\omega^t) \min_{j=1,\dots,N} \left\{\Psi_{t+1}\left(\omega^t \otimes_t z_j, (a^{t},\widetilde{a})\right)+\lambda \|\mathcal{R}_{i+1} -z_j\|\right\}\right] - \lambda \varepsilon \right)  \right)^2
$$
 w.r.t. parameters of $\tilde{\Psi}_t$:

Finally, we set:
$$
\Omega^t \ni \omega^t =(\omega_1,\dots,\omega_{t}) \mapsto a_t^*(\omega^t):= \widetilde{a}_t^*\left(\omega^t,~\left(a_0^*,\dots,a_{t-1}^*(\omega_1,\dots,\omega_{t-1}\right)\right) \in A.
$$

# Training

Number of periods

In [ ]:
T = 10

Initialize the dictionaries

Set the parameters

In [ ]:
inner_psi = 2000
inner_a = 500
tolerance_psi = 0.0001
tolerance_a = 0.00001
learning_rate_Psi = 0.001
learning_rate_a = 0.001
learning_rate_LAM = 0.001
# Number of Monte Carlo Simulations
N_MC = 128
N_MC_inner = 128
#Batch Sizes
Batch_size_a = 128
Batch_size_psi = 128

Train

In [ ]:
#compute
print("Start optimization")
print("Non-Robust")
a_tilde_5dim, _ = train_networks_ndim(Returns_train_tf,T=T,inner_psi = inner_psi,
                   inner_a =inner_a, N_MC=N_MC, N_MC_inner=N_MC_inner,tolerance_psi =tolerance_psi,
                   tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                    learning_rate_LAM = learning_rate_LAM, Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi,print_intermediate_results = True) 
#save
file_save(a_tilde_5dim,"Basket","Non-Robust_5dim")

#compute
print("Start optimization")
print("Robust, epsilon = 0.0001")
a_tilde_5dim_0001, _ = train_networks_ndim(Returns_train_tf,T=T,inner_psi = inner_psi,
                   inner_a =inner_a, N_MC=N_MC, N_MC_inner=N_MC_inner,
                   epsilon = 0.0001,tolerance_psi =tolerance_psi,
                   tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                    learning_rate_LAM = learning_rate_LAM,Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi,print_intermediate_results = True)   
#save
file_save(a_tilde_5dim_0001,"Basket","Robust_5dim_0001")



#compute
print("Start optimization")
print("Robust, epsilon = 0.001")
a_tilde_5dim_001, _ = train_networks_ndim(Returns_train_tf,T=T,inner_psi = inner_psi,
                   inner_a =inner_a,  N_MC=N_MC, N_MC_inner=N_MC_inner, epsilon = 0.001,tolerance_psi =tolerance_psi,
                   tolerance_a = tolerance_a, learning_rate_Psi = learning_rate_Psi,learning_rate_a = learning_rate_a,
                                    learning_rate_LAM = learning_rate_LAM, Batch_size_a = Batch_size_a,
                                        Batch_size_psi = Batch_size_psi)  
#save
file_save(a_tilde_5dim_001,"Basket","Robust_5dim_001")